In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import sqlite3
from datetime import date
from datetime import datetime as dt
import hashlib
import sys
import os
import locale
import re
import requests as rq
import wbdata as wb
from contextlib import contextmanager
import pytz
from datetime import timedelta

project_dir = '/home/mi/Dropbox/prog/python/tradeDB'
os.chdir(project_dir)

from importlib import reload
from workers import sql
from workers import api
from workers.common import read_json
import tradeDB

In [2]:
SECTORS = {
            "INDEXES": {"file": "./assets/indexes.jsonc"},
            "STOCK": {"file": "./assets/stock.jsonc"},
            "ETF": {"file": "./assets/etf.jsonc"},
            "COMODITIES": {"file": "./assets/comodities.jsonc"},
        }
for k in SECTORS:
    SECTORS[k]["data"] = read_json( # type: ignore
        SECTORS[k]["file"]
    )

In [405]:
trade= tradeDB.Trader()
db_file = trade.db

In [406]:
reload(sql)

<module 'workers.sql' from '/home/mi/Dropbox/prog/python/tradeDB/workers/sql.py'>

In [410]:
dat = pd.read_csv('./dev/dat.csv')
tab='STOCK'
tabL=[tab+'_DESC',tab]

In [431]:
# merge with what we already know
known = sql.query(
        db_file=db_file,
        tab=tab,
        symbol=dat.loc[:, "symbol"].tolist(),
        from_date=date(1900,1,1),
        to_date=date.today()
    )
# add hash column
known['hash']=sql.get(tab=tabL[0],
        get=['hash'],
        search=dat.loc[:, "symbol"].tolist(),
        db_file=db_file,
        cols=['symbol'])['symbol']


In [432]:
known

,symbol,country,from_date,info,date,vol,name,to_date,industry,val,hash
0,ACP,PL,2023-10-06,None,2023-10-06,None,ASSECOPOL,2023-10-06,None,74.950,bf66636c7010aa2b3611165c364fac17
1,ALE,PL,2023-10-06,None,2023-10-06,None,ALLEGRO,2023-10-06,None,28.750,81e72552fe1dca121c25c8e83cc53c49
2,ALR,PL,2023-10-06,None,2023-10-06,None,ALIOR,2023-10-06,None,55.740,828491131bf307a86f8cda051f6ac09b
3,CDR,PL,2023-10-06,None,2023-10-06,None,CDPROJEKT,2023-10-06,None,107.600,b263eda51983feabe090a9a11f13fe4d
4,CPS,PL,2023-10-06,None,2023-10-06,None,CYFRPLSAT,2023-10-06,None,11.430,0fedeccaf3d740fedec223ea72c1aa20
5,DNP,PL,2023-10-06,None,2023-10-06,None,DINOPL,2023-10-06,None,350.800,b2af1a6920faa8b01d129712b146c1ed
6,JSW,PL,2023-10-06,None,2023-10-06,None,JSW,2023-10-06,None,40.580,d39f06f94b143236a70481655eed5e52
7,KGH,PL,2023-10-06,None,2023-10-06,None,KGHM,2023-10-06,None,107.400,ba7c40134b5d54b8d0ad86aeb9ae8043
8,KRU,PL,2023-10-06,None,2023-10-06,None,KRUK,2023-10-06,None,406.600,24b0c5cea875ec019eda3409c13881e5
9,KTY,PL,2023-10-06,None,2023-10-06,None,KETY,2023-10-06,None,630.000,0647b7c8a8d0c3ae8f6de8ea65ee5d8f


In [433]:
dat.loc[:,[c in known.columns for c in dat.columns]]

,symbol,name,val,date,country,hash,from_date,to_date
0,ACP,ASSECOPOL,74.950,2023-10-06,PL,bf66636c7010aa2b3611165c364fac17,2023-10-06,2023-10-06
1,ALE,ALLEGRO,120.000,2023-10-06,PL,81e72552fe1dca121c25c8e83cc53c49,2023-10-06,2023-10-06
2,ALR,ALIOR,55.740,2023-10-06,PL,828491131bf307a86f8cda051f6ac09b,2023-10-06,2023-10-06
3,CDR,CDPROJEKT,107.600,2023-10-06,PL,b263eda51983feabe090a9a11f13fe4d,2023-10-06,2023-10-06
4,CPS,CYFRPLSAT,11.430,2023-10-06,PL,0fedeccaf3d740fedec223ea72c1aa20,2023-10-06,2023-10-06
5,DNP,DINOPL,350.800,2023-10-06,PL,b2af1a6920faa8b01d129712b146c1ed,2023-10-06,2023-10-06
6,JSW,JSW,40.580,2023-10-06,PL,d39f06f94b143236a70481655eed5e52,2023-10-06,2023-10-06
7,KGH,KGHM,107.400,2023-10-06,PL,ba7c40134b5d54b8d0ad86aeb9ae8043,2023-10-06,2023-10-06
8,KRU,KRUK,406.600,2023-10-06,PL,24b0c5cea875ec019eda3409c13881e5,2023-10-06,2023-10-06
9,KTY,KETY,630.000,2023-10-06,PL,0647b7c8a8d0c3ae8f6de8ea65ee5d8f,2023-10-06,2023-10-06
